# Homework 6

In this assignment you'll be building a "dashboard" interface for the buildings dataset which provides the user with a linked view of this dataset.  The purpose of this assignment is to get some "hands-on" practice linking different facets of a dataset in a dashboard to provide the user a way to interactively browse their data.

Build a dashboard for the buildings data (see CSV file attached).

    Left component:
        Grid heat map
        Rows are congressional district
        Columns are the governmental department (Agency Name) (note, the agency names might overlap with your plot -- there are ways around this or you can leave as is)
        Values are sum of total square footage for that set of criteria
    Right component:
        line plot
        x is the year
        y is total square footage acquired that year
    These two should be linked so that you can select cells and that will update the square footage plot.
    Things to think about:
        Can you keep the x and y ranges static on the line plot?
        Can you change the style?

In your write-up please address:

    Any data transformations or rescalings you did for your plot
    How you dealt with NaN's (empty entries) in your dataset
    Any aesthetic choices you made (colors, layout, plot size, label size) and why, and what you'd like to experiment with if you had more time.


In [194]:
#%matplotlib widgets
# this add more fine interactivity

import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import ipywidgets
import bqplot

https://bqplot.readthedocs.io/en/latest/

In [195]:
buildings = pd.read_csv("building_inventory.csv",
           na_values = {'Square Footage': 0,
                       'Year Acquired': 0,
                       'Year Constructed': 0}) 

In [196]:
buildings # data for this assignment

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975.0,1975.0,144.0,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8857,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte. 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,In Use,NaN,NaN,432.0,1,0,0,Storage,NaN,NaN
8858,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,In Use,NaN,NaN,330.0,1,0,0,Storage,NaN,NaN
8859,Department of Transportation,Quincy Maintenance Storage Facility,800 Koch's Lane,Quincy,62305,Adams,18,Darin M. LaHood,94,Frese Randy E.,...,In Use,NaN,1987.0,130.0,1,0,0,Storage,High Hazard,NaN
8860,Illinois Community College Board,Illinois Valley Community College - Oglesby,815 North Orlando Smith Avenue,Oglesby,61348,LaSalle,16,Adam Kinzinger,76,Long Jerry Lee,...,In Use,1971.0,1971.0,49552.0,1,1,0,Education,Education,Not provided


In [197]:
buildings.shape # give us the total rows and columns

(8862, 22)

In [198]:
buildings.info() # gives us the information about the data frame (e.g., column names, data-types)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8862 entries, 0 to 8861
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Agency Name              8862 non-null   object 
 1   Location Name            8862 non-null   object 
 2   Address                  8811 non-null   object 
 3   City                     8862 non-null   object 
 4   Zip code                 8862 non-null   int64  
 5   County                   8837 non-null   object 
 6   Congress Dist            8862 non-null   int64  
 7   Congressional Full Name  8699 non-null   object 
 8   Rep Dist                 8862 non-null   int64  
 9   Rep Full Name            8839 non-null   object 
 10  Senate Dist              8862 non-null   int64  
 11  Senator Full Name        8839 non-null   object 
 12  Bldg Status              8862 non-null   object 
 13  Year Acquired            8597 non-null   float64
 14  Year Constructed        

The following are attempts of me parsing the data to have the each agency name and congress district seperated and then joing into a grid. The problem with this for a grid is that the arrays/series for each are different length. Our grid will have empty squares since not all congress districts have the same agencies in them. 

In [199]:
# the idea here is to make an array manually by separating the groups into their districts and then combining them all into a series
# the problem that i see here is that some might have more info than others
congress_dist_1 = buildings.groupby('Congress Dist').get_group(0)['Agency Name']
congress_dist_1

101          Department of Natural Resources
102          Department of Natural Resources
149     Illinois Medical District Commission
414          Department of Natural Resources
430               Department of State Police
                        ...                 
8282    Illinois Medical District Commission
8311    Illinois Emergency Management Agency
8640            Department of Transportation
8665            Department of Transportation
8731            Department of Transportation
Name: Agency Name, Length: 163, dtype: object

In [200]:
# another failed attempted at getting the data necessary for the grid
count = 0
array = []
for i in range(buildings['Congress Dist'].nunique()):
    agency_list = list(buildings.groupby('Congress Dist').get_group(count)['Agency Name'])
    array.append(agency_list)
    count += 1
    
len(array)

19

In [201]:
congress_agency = buildings.groupby('Congress Dist')[['Congress Dist','Agency Name']]
type(congress_agency)

pandas.core.groupby.generic.DataFrameGroupBy


### Declarative / Grammar of Graphics flow

1. Data
2. Scales
3. Axis
4. Marks
5. Interaction
6. Figure


*Marks and Axis are interchangeable in the flow since they're independant of one another*


## Grid Heat Map 

**Step 1.** Data

In [202]:
bqplot?

In [203]:
# I had a difficult time parsing the data from the table but on 20210302 I went to office hours and the ta 
# recommended I make a new dataframe and use the Congress Dist as the index
# the same strategy can be done for the line graph
buildings_heatmap = pd.pivot_table(buildings, index=["Congress Dist"],values=["Square Footage"],
                                 columns=["Agency Name"],
                                  aggfunc=np.sum)


In [204]:
buildings_heatmap

Square Footage  \
Agency Name   Appellate Court / Fifth District   
Congress Dist                                    
0                                          NaN   
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   
5                                          NaN   
6                                          NaN   
7                                          NaN   
8                                          NaN   
9                                          NaN   
10                                         NaN   
11                                         NaN   
12                                     15124.0   
13                                         NaN   
14                                         NaN   
15                                         NaN   
16                                         NaN   
17                                         NaN   
18                                         NaN   

                                                 \
Agency Name   Appellate Court / Fourth District   
Congress Dist                                     
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   
5                                           NaN   
6                                           NaN   
7                                           NaN   
8                                           NaN   
9                                           NaN   
10                                          NaN   
11                                          NaN   
12                                          NaN   
13                                      16400.0   
14                                          NaN   
15                                          NaN   
16                                          NaN   
17                                          NaN   
18                                          NaN   

                                                 \
Agency Name   Appellate Court / Second District   
Congress Dist                                     
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   
5                                           NaN   
6                                           NaN   
7                                           NaN   
8                                       43330.0   
9                                           NaN   
10                                          NaN   
11                                          NaN   
12                                          NaN   
13                                          NaN   
14                                          NaN   
15                                          NaN   
16                                          NaN   
17                                          NaN   
18                                          NaN   

                                                                         \
Agency Name   Appellate Court / Third District Chicago State University   
Congress Dist                                                             
0                                          NaN                      NaN   
1                                          NaN                1219492.0   
2                                          NaN                      NaN   
3                                          NaN                      NaN   
4                                          NaN                      NaN   
5                                          NaN                      NaN   
6         

In [205]:
buildings_heatmap.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], dtype='int64', name='Congress Dist')

In [206]:
b = buildings_heatmap.columns.levels[1].to_list()
b

['Appellate Court / Fifth District',
 'Appellate Court / Fourth District',
 'Appellate Court / Second District',
 'Appellate Court / Third District',
 'Chicago State University',
 'Department of Agriculture',
 'Department of Central Management Services',
 'Department of Corrections',
 'Department of Human Services',
 'Department of Juvenile Justice',
 'Department of Military Affairs',
 'Department of Natural Resources',
 'Department of Public Health',
 'Department of Revenue',
 'Department of State Police',
 'Department of Transportation',
 "Department of Veterans' Affairs",
 'Eastern Illinois University',
 "Governor's Office",
 'Governors State University',
 'Historic Preservation Agency',
 'IL State Board of Education',
 'Illinois Board of Higher Education',
 'Illinois Community College Board',
 'Illinois Courts',
 'Illinois Emergency Management Agency',
 'Illinois Medical District Commission',
 'Illinois State University',
 'Northeastern Illinois University',
 'Northern Illinois Uni

Another way we can do this instead of a pivot table is we can do it by hand like in class generating our own 2d histogram and get the aggregation by hand and select for a unique congress dist and agency name and aggregate the square footage avoiding doing aggregation by hand

**Step 1.** Data

In [207]:
color_data = buildings_heatmap.values
# this doesnt work buildings.groupby('Congress Dist')[['Agency Name','Square Footage']].sum()# buildings['Square Footage'].sum()
row_data = buildings_heatmap.index # or buildings['Congress Dist'].unique()
column_data = buildings_heatmap.columns.levels[1].to_list()  # or buildings['Agency Name'].unique()
color_data
# why doesn this work? # can you exbuildings_heatmap.columns.values

array([[          nan,           nan,           nan,           nan,
                  nan,           nan, 2.3127800e+05,           nan,
        3.7278400e+05,           nan,           nan, 4.0249000e+04,
                  nan,           nan, 8.2390000e+03, 5.5578000e+04,
        3.0000000e+03,           nan,           nan,           nan,
        3.8310000e+03,           nan,           nan,           nan,
                  nan, 5.6500000e+03, 3.1200000e+04,           nan,
                  nan, 4.1315000e+04,           nan,           nan,
                  nan,           nan,           nan],
       [          nan,           nan,           nan,           nan,
        1.2194920e+06,           nan,           nan,           nan,
        4.4954700e+05,           nan, 5.6458000e+05,           nan,
                  nan,           nan,           nan, 1.1814300e+05,
                  nan,           nan,           nan,           nan,
                  nan,           nan,           nan,          

**Step 2.** Scales

In [208]:
bqplot.scales?

In [209]:
# color scale
col_sc = bqplot.ColorScale()
# x/y scale
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

**Step 3.** Axis

In [210]:
bqplot.Axis?

In [211]:
# color axis 
c_ax = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right')
# x/y axis
ax_x = bqplot.Axis(scale = x_sc, label = 'Agency Name') 
ax_y = bqplot.Axis(scale = y_sc, label = 'Congress District', orientation = 'vertical')

**Step 4.** Marks

In [212]:
bqplot.marks?

In [213]:
bqplot.GridHeatMap?

In [214]:
heat_map = bqplot.GridHeatMap(color=np.log10(buildings_heatmap.values), row=row_data, column=column_data , scales={'color':col_sc, 'row':y_sc, 'column':x_sc})
# marking data we have on top and combining it with the scales we made

In [215]:
heat_map

GridHeatMap(color=array([[       nan,        nan,        nan,        nan,        nan,
               nan, 5.36413432,        nan, 5.57145726,        nan,
               nan, 4.60475509,        nan,        nan, 3.9158745 ,
        4.74490291, 3.47712125,        nan,        nan,        nan,
        3.58331215,        nan,        nan,        nan,        nan,
        3.75204845, 4.49415459,        nan,        nan, 4.61610776,
               nan,        nan,        nan,        nan,        nan],
       [       nan,        nan,        nan,        nan, 6.08617896,
               nan,        nan,        nan, 5.6527751 ,        nan,
        5.75172549,        nan,        nan,        nan,        nan,
        5.07240799,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan],
       [       nan,        n

**Step 4.** Interaction

**Step 5.** Figure

In [216]:
bqplot.Figure?

In [217]:
fig = bqplot.Figure(title='Grid Heat Map between Congressional District and Agency Name', axes= [c_ax, ax_x, ax_y], marks=[heat_map])

In [218]:
fig

Figure(axes=[ColorAxis(orientation='vertical', scale=ColorScale(), side='right'), Axis(label='Agency Name', sc…

## Line Plot

Right component:
        line plot
        x is the year
        y is total square footage acquired that year

In [219]:
buildings

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975.0,1975.0,144.0,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8857,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte. 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,In Use,NaN,NaN,432.0,1,0,0,Storage,NaN,NaN
8858,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,In Use,NaN,NaN,330.0,1,0,0,Storage,NaN,NaN
8859,Department of Transportation,Quincy Maintenance Storage Facility,800 Koch's Lane,Quincy,62305,Adams,18,Darin M. LaHood,94,Frese Randy E.,...,In Use,NaN,1987.0,130.0,1,0,0,Storage,High Hazard,NaN
8860,Illinois Community College Board,Illinois Valley Community College - Oglesby,815 North Orlando Smith Avenue,Oglesby,61348,LaSalle,16,Adam Kinzinger,76,Long Jerry Lee,...,In Use,1971.0,1971.0,49552.0,1,1,0,Education,Education,Not provided


In [220]:
buildings.groupby('Year Acquired')['Square Footage'].sum()
# series with shape (171,) which idk what that means a 1D series??

Year Acquired
1753.0      1200.0
1802.0      4440.0
1810.0      4033.0
1832.0    120000.0
1837.0     10302.0
            ...   
2015.0    305093.0
2016.0    304839.0
2017.0      6720.0
2018.0     17160.0
2019.0      1520.0
Name: Square Footage, Length: 171, dtype: float64

In [221]:
# thought i could make a new data frame like in the grid before for the line plot
# for line plot this is uncesserary
# using groupby and then .index and .values works
buildings_line = pd.pivot_table(buildings, index=["Year Acquired"],values=["Square Footage"],
                                 columns=["Square Footage"],
                                  aggfunc=np.sum)

ValueError: Grouper for 'Square Footage' not 1-dimensional

In [222]:
bqplot.Lines?

In [223]:
# 1. data

# year = buildings.groupby('Year Acquired')['Square Footage'] # not needed. I can use .index to get the years
total_square_footage = buildings.groupby('Year Acquired')['Square Footage'].sum()

# 2. Scales

x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

# 3. Axes

ax_x = bqplot.Axis(scale=x_sc, label='Year Acquired')
ax_y = bqplot.Axis(scale=y_sc, label='Total Square Footage', orientation='vertical')


# 4. Marks

lines = bqplot.Lines(x=total_square_footage.index, y=total_square_footage.values, scales={'x':x_sc, 'y':y_sc})

# 5. Interactivity


# 6. Figure
line_fig = bqplot.Figure(marks=[lines], axes=[ax_x, ax_y])
line_fig



Figure(axes=[Axis(label='Year Acquired', scale=LinearScale()), Axis(label='Total Square Footage', orientation=…

In [224]:
total_square_footage.values

array([1.200000e+03, 4.440000e+03, 4.033000e+03, 1.200000e+05,
       1.030200e+04, 2.010000e+03, 1.175200e+04, 3.691800e+04,
       3.256000e+03, 9.000000e+02, 4.088000e+03, 6.584600e+04,
       2.129200e+04, 2.267000e+03, 2.550000e+05, 2.140000e+03,
       6.615400e+04, 7.500000e+02, 1.812400e+04, 2.236700e+04,
       4.847000e+03, 2.741700e+04, 1.579500e+04, 2.234940e+05,
       2.700000e+03, 2.864000e+03, 6.403000e+03, 1.490000e+04,
       1.100000e+04, 6.600000e+03, 3.808000e+03, 4.042800e+05,
       3.307100e+04, 1.798330e+05, 6.671000e+03, 5.926600e+04,
       8.725000e+03, 3.015700e+04, 1.683700e+05, 1.948230e+05,
       8.492400e+04, 8.633400e+04, 2.325000e+03, 1.514800e+05,
       3.959000e+04, 8.500000e+04, 2.256770e+05, 2.901650e+05,
       1.329070e+05, 3.676000e+04, 1.214750e+05, 3.358350e+05,
       1.094860e+05, 2.697250e+05, 1.605840e+05, 4.096400e+04,
       3.128860e+05, 2.037400e+04, 1.332850e+05, 3.602520e+05,
       2.982830e+05, 2.314110e+05, 2.377220e+05, 4.2281

In [225]:
# 1. data --  buildings_heatmap
buildings_heatmap = pd.pivot_table(buildings, index=["Congress Dist"],values=["Square Footage"],
                                 columns=["Agency Name"],
                                  aggfunc=np.sum)
color_data = buildings_heatmap.values
congress_dist = buildings_heatmap.index # or buildings['Congress Dist'].unique()
agency_name = buildings_heatmap.columns.levels[1].to_list()  # or buildings['Agency Name'].unique()

# 2. scales
# 2.1 color scale
grid_col_sc = bqplot.ColorScale(scheme='Blues')
# 2.2 x/y scales
grid_x_sc = bqplot.OrdinalScale() # categorical
grid_y_sc = bqplot.OrdinalScale()

# 3. Axis
# 3.1 color axis means colorbar
grid_c_ax = bqplot.ColorAxis(scale=grid_col_sc, orientation='vertical', side='right')
# 3.2 x/y axis
grid_x_ax = bqplot.Axis(scale=grid_x_sc)
grid_y_ax = bqplot.Axis(scale=grid_y_sc, orientation='vertical')

# 4. Mark
grid_heat_map = bqplot.GridHeatMap(color=np.log10(color_data), row=congress_dist, column=agency_name , scales={'color':grid_col_sc, 'row':grid_y_sc, 'column':grid_x_sc},
                              interactions={'click':'select'}, # if i click on a bin it selects that bin
                              anchor_style={'fill':'red'}, # this highlights our selection to the color red
                              selected_style={'fill':'red'}, # changes our selected box a bit
                              unselected_style={'opacity':1.0} # a lil see through for the not selected ones
                              )


# 5. Interaction ... adding to the 'marks' w/ interactions


# 6. Figure
fig = bqplot.Figure(title='Grid Heat Map between Congressional District and Agency Name', axes= [grid_c_ax, grid_x_ax, grid_y_ax], marks=[grid_heat_map])
fig

Figure(axes=[ColorAxis(orientation='vertical', scale=ColorScale(scheme='Blues'), side='right'), Axis(scale=Ord…

## Building interaction

Grid map

In [226]:
grid_heat_map.selected # the trait of x/y indices of the heat map gets 

array([[12,  7]])

In [227]:
def on_selected(change):
    print (change)

In [228]:
# 1. data --  buildings_heatmap
buildings_heatmap = pd.pivot_table(buildings, index=["Congress Dist"],values=["Square Footage"],
                                 columns=["Agency Name"],
                                  aggfunc=np.sum)
color_data = buildings_heatmap.values
congress_dist = buildings_heatmap.index # or buildings['Congress Dist'].unique()
agency_name = buildings_heatmap.columns.levels[1].to_list()  # or buildings['Agency Name'].unique()

# 2. scales
# 2.1 color scale
grid_col_sc = bqplot.ColorScale(scheme='Blues')
# 2.2 x/y scales
grid_x_sc = bqplot.OrdinalScale() # categorical
grid_y_sc = bqplot.OrdinalScale()

# 3. Axis
# 3.1 color axis means colorbar
grid_c_ax = bqplot.ColorAxis(scale=grid_col_sc, orientation='vertical', side='right')
# 3.2 x/y axis
grid_x_ax = bqplot.Axis(scale=grid_x_sc)
grid_y_ax = bqplot.Axis(scale=grid_y_sc, orientation='vertical')

# 4. Mark
grid_heat_map = bqplot.GridHeatMap(color=np.log10(color_data), row=congress_dist, column=agency_name , scales={'color':grid_col_sc, 'row':grid_y_sc, 'column':grid_x_sc},
                              interactions={'click':'select'}, # if i click on a bin it selects that bin
                              anchor_style={'fill':'red'}, # this highlights our selection to the color red
                              selected_style={'fill':'red'}, # changes our selected box a bit
                              unselected_style={'opacity':1.0} # a lil see through for the not selected ones
                              )


# 5. Interaction ... adding to the 'marks' w/ interactions, adding observe func
grid_heat_map.observe(on_selected, 'selected')


# 6. Figure
fig = bqplot.Figure(title='Grid Heat Map between Congressional District and Agency Name', axes= [grid_c_ax, grid_x_ax, grid_y_ax], marks=[grid_heat_map])
fig

Figure(axes=[ColorAxis(orientation='vertical', scale=ColorScale(scheme='Blues'), side='right'), Axis(scale=Ord…

updating our function in step 5 (change['owner'].selected)

In [229]:
# 1. data --  buildings_heatmap
buildings_heatmap = pd.pivot_table(buildings, index=["Congress Dist"],values=["Square Footage"],
                                 columns=["Agency Name"],
                                  aggfunc=np.sum)
color_data = buildings_heatmap.values
congress_dist = buildings_heatmap.index # or buildings['Congress Dist'].unique()
agency_name = buildings_heatmap.columns.levels[1].to_list()  # or buildings['Agency Name'].unique()

# 2. scales
# 2.1 color scale
col_sc = bqplot.ColorScale(scheme='Blues')
# 2.2 x/y scales
x_sc = bqplot.OrdinalScale() # categorical
y_sc = bqplot.OrdinalScale()

# 3. Axis
# 3.1 color axis means colorbar
c_ax = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right')
# 3.2 x/y axis
x_ax = bqplot.Axis(scale=x_sc)
y_ax = bqplot.Axis(scale=y_sc, orientation='vertical')

# 4. Mark
heat_map = bqplot.GridHeatMap(color=np.log10(color_data), row=congress_dist, column=agency_name , scales={'color':col_sc, 'row':y_sc, 'column':x_sc},
                              interactions={'click':'select'}, # if i click on a bin it selects that bin
                              anchor_style={'fill':'red'}, # this highlights our selection to the color red
                              selected_style={'fill':'red'}, # changes our selected box a bit
                              unselected_style={'opacity':1.0} # a lil see through for the not selected ones
                              )


# 5. Interaction ... adding to the 'marks' w/ interactions
def on_selected(change):
    if len(change['owner'].selected) == 1:
        print (change['owner'].selected) # returns the row and column indecies and it only lets people select one box
    
heat_map.observe(on_selected, 'selected')

# 6. Figure
fig = bqplot.Figure(title='Grid Heat Map between Congressional District and Agency Name', axes= [c_ax, x_ax, y_ax], marks=[heat_map])
fig

Figure(axes=[ColorAxis(orientation='vertical', scale=ColorScale(scheme='Blues'), side='right'), Axis(scale=Ord…

In [230]:
# making a label for the selected stuff with ipywidgets
mySelectedLabel = ipywidgets.Label()
mySelectedLabel.value = 'thing'

In [231]:
# 1. data --  buildings_heatmap
buildings_heatmap = pd.pivot_table(buildings, index=["Congress Dist"],values=["Square Footage"],
                                 columns=["Agency Name"],
                                  aggfunc=np.sum)
color_data = buildings_heatmap.values
congress_dist = buildings_heatmap.index # or buildings['Congress Dist'].unique()
agency_name = buildings_heatmap.columns.levels[1].to_list()  # or buildings['Agency Name'].unique()

# 2. scales
# 2.1 color scale
col_sc = bqplot.ColorScale(scheme='Blues')
# 2.2 x/y scales
x_sc = bqplot.OrdinalScale() # categorical
y_sc = bqplot.OrdinalScale()

# 3. Axis
# 3.1 color axis means colorbar
c_ax = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right')
# 3.2 x/y axis
x_ax = bqplot.Axis(scale=x_sc)
y_ax = bqplot.Axis(scale=y_sc, orientation='vertical')

# 4. Mark
heat_map = bqplot.GridHeatMap(color=np.log10(color_data), row=congress_dist, column=agency_name , scales={'color':col_sc, 'row':y_sc, 'column':x_sc},
                              interactions={'click':'select'}, # if i click on a bin it selects that bin
                              anchor_style={'fill':'red'}, # this highlights our selection to the color red
                              selected_style={'fill':'red'}, # changes our selected box a bit
                              unselected_style={'opacity':1.0} # a lil see through for the not selected ones
                              )


# 5. Interaction ... adding to the 'marks' w/ interactions
def on_selected(change):
    if len(change['owner'].selected) == 1:
        print (change['owner'].selected) # returns the row and column indecies and it only lets people select one box
    
heat_map.observe(on_selected, 'selected')

# 6. Figure
fig = bqplot.Figure(title='Grid Heat Map between Congressional District and Agency Name', axes= [c_ax, x_ax, y_ax], marks=[heat_map])
#fig

myDashboard = ipywidgets.VBox([mySelectedLabel, fig])
myDashboard

starting to think about the i, j and how I can use these values to get subset and connect them to the line plot

In [232]:
buildings_heatmap.iloc[1,4] # the square footage sum of the district 1 and whatever the 4th index agency is

1219492.0

In [233]:
buildings_heatmap.columns[4] # cxhecking to see what the 4'th index column returns. brings back the square footage and agency name

('Square Footage', 'Chicago State University')

In [234]:
# buildings_heatmap.columns[4]
buildings_heatmap.columns[4][1] 
# returns the 4th indexed agency name... a str is returned
# the 0th index is 'square footage'

'Chicago State University'

Updating what out label says. Adding square footage, agency name, and congress district to the selected

In [235]:
# 1. data --  buildings_heatmap
buildings_heatmap = pd.pivot_table(buildings, index=["Congress Dist"],values=["Square Footage"],
                                 columns=["Agency Name"],
                                  aggfunc=np.sum)
color_data = buildings_heatmap.values
congress_dist = buildings_heatmap.index # or buildings['Congress Dist'].unique()
agency_name = buildings_heatmap.columns.levels[1].to_list()  # or buildings['Agency Name'].unique()

# 2. scales
# 2.1 color scale
col_sc = bqplot.ColorScale(scheme='Blues')
# 2.2 x/y scales
x_sc = bqplot.OrdinalScale() # categorical
y_sc = bqplot.OrdinalScale()

# 3. Axis
# 3.1 color axis means colorbar
c_ax = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right')
# 3.2 x/y axis
x_ax = bqplot.Axis(scale=x_sc)
y_ax = bqplot.Axis(scale=y_sc, orientation='vertical')

# 4. Mark
heat_map = bqplot.GridHeatMap(color=np.log10(color_data), row=congress_dist, column=agency_name , scales={'color':col_sc, 'row':y_sc, 'column':x_sc},
                              interactions={'click':'select'}, # if i click on a bin it selects that bin
                              anchor_style={'fill':'red'}, # this highlights our selection to the color red
                              selected_style={'fill':'red'}, # changes our selected box a bit
                              unselected_style={'opacity':1.0} # a lil see through for the not selected ones
                              )


# 5. Interaction ... adding to the 'marks' w/ interactions
def on_selected(change):
    if len(change['owner'].selected) == 1:
        i, j = change['owner'].selected[0] # why is index 0 here? wouldnt this make i and j the same value as the row rather than the row and column index of the grid
        # i represents that congress dist/ row and j is the agency index number
        v = buildings_heatmap.iloc[i,j] # value of the quare footage in that location
        mySelectedLabel.value = 'Square footage: ' + str(v) + '| Agency Name: ' + buildings_heatmap.columns[i][1] + '| Congress District: ' + str(i) 
        
heat_map.observe(on_selected, 'selected')

# 6. Figure
fig = bqplot.Figure(title='Grid Heat Map between Congressional District and Agency Name', axes= [c_ax, x_ax, y_ax], marks=[heat_map])
#fig

myDashboard = ipywidgets.VBox([mySelectedLabel, fig])
myDashboard

## Linking line plot with grid heat map

In [236]:
# reminder of the line plot

# Line plot
# 1. data

# year = buildings.groupby('Year Acquired')['Square Footage']
total_square_footage = buildings.groupby('Year Acquired')['Square Footage'].sum()
# maybe we can use get group to subset our line plot data
# groupby('Year Acquire').get_group(buildings_heatmap.columns[i][1])['Square Footage'].sum() 

# 2. Scales

x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

# 3. Axes

ax_x = bqplot.Axis(scale=x_sc, label='Year Acquired')
ax_y = bqplot.Axis(scale=y_sc, label='Total Square Footage', orientation='vertical')


# 4. Marks

lines = bqplot.Lines(x=total_square_footage.index, y=total_square_footage.values, scales={'x':x_sc, 'y':y_sc})

# 5. Interactivity


# 6. Figure
line_fig = bqplot.Figure(marks=[lines], axes=[ax_x, ax_y])
line_fig



Figure(axes=[Axis(label='Year Acquired', scale=LinearScale()), Axis(label='Total Square Footage', orientation=…

The next few boxes is me thinking of how i can use groupby and get_group to parse my data. the numbers are place holders for the indices that the grid heat map can get

In [237]:
agency_name = buildings.groupby(['Agency Name']).get_group(buildings_heatmap.columns[4][1])
agency_name

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
418,Chicago State University,Chicago State University - Cook County,95th & King Dirve,Chicago,60628,Cook,1,Bobby L. Rush,34,"Sims, Jr. Elgie R.",...,In Use,2005.0,2005.0,140000.0,6,5,1,Business,Business,Not provided
587,Chicago State University,Chicago State University - Cook County,95th & King Dirve,Chicago,60628,Cook,1,Bobby L. Rush,34,"Sims, Jr. Elgie R.",...,In Use,2005.0,2005.0,172000.0,5,4,1,Assembly,Assembly,Not provided
792,Chicago State University,Chicago State University - Cook County,95th & King Dirve,Chicago,60628,Cook,1,Bobby L. Rush,34,"Sims, Jr. Elgie R.",...,In Progress,2010.0,2010.0,30000.0,2,2,0,Education,Education,Not provided
1098,Chicago State University,Chicago State University - Cook County,95th & King Dirve,Chicago,60628,Cook,1,Bobby L. Rush,34,"Sims, Jr. Elgie R.",...,In Progress,2010.0,2010.0,37000.0,3,2,1,Business,Business,Not provided
7932,Chicago State University,Chicago State University - Cook County,95th & King Dirve,Chicago,60628,Cook,1,Bobby L. Rush,34,"Sims, Jr. Elgie R.",...,In Use,1973.0,1973.0,70995.0,7,6,1,Education,Education,Not provided
7933,Chicago State University,Chicago State University - Cook County,95th & King Dirve,Chicago,60628,Cook,1,Bobby L. Rush,34,"Sims, Jr. Elgie R.",...,In Use,1971.0,1971.0,70995.0,7,6,1,Education,Education,Not provided
7934,Chicago State University,Chicago State University - Cook County,95th & King Dirve,Chicago,60628,Cook,1,Bobby L. Rush,34,"Sims, Jr. Elgie R.",...,In Use,1973.0,1973.0,113164.0,6,5,1,Education,Business,Not provided
7935,Chicago State University,Chicago State University - Cook County,95th & King Dirve,Chicago,60628,Cook,1,Bobby L. Rush,34,"Sims, Jr. Elgie R.",...,In Use,1973.0,1972.0,185458.0,6,5,1,Assembly,Assembly,Not provided
7936,Chicago State University,Chicago State University - Cook County,95th & King Dirve,Chicago,60628,Cook,1,Bobby L. Rush,34,"Sims, Jr. Elgie R.",...,In Use,1970.0,1970.0,66968.0,6,5,1,Business,Business,Not provided
7937,Chicago State University,Chicago State University - Cook County,95th & King Dirve,Chicago,60628,Cook,1,Bobby L. Rush,34,"Sims, Jr. Elgie R.",...,In Use,1970.0,1970.0,116800.0,5,4,1,Education,Assembly,Not provided


In [238]:
hello = buildings.groupby(['Agency Name']).get_group('Department of Natural Resources')[['Year Acquired', 'Square Footage']]
df_test = hello.groupby('Year Acquired').sum()
df_test
# what i learned from this is you can group by something then parse it to a distinct group 
# and then groupby that data frame

,Square Footage
Year Acquired,
1840.0,760.0
1841.0,3256.0
1848.0,1644.0
1875.0,1428.0
1876.0,1200.0
...,...
2010.0,288.0
2011.0,1200.0
2013.0,432.0


In [239]:
hello = buildings.groupby(['Agency Name']).get_group(buildings_heatmap.columns[i][1])[['Year Acquired', 'Square Footage']]
df = hello.groupby('Year Acquired').sum()
df # the quare footage total for an agency 

,Square Footage
Year Acquired,
1855.0,45120.0


In [240]:
type(buildings.groupby(['Agency Name']).get_group(buildings_heatmap.columns[i][1]))

pandas.core.frame.DataFrame

In [241]:
df = buildings.groupby(['Agency Name', 'Year Acquired']).agg({'Square Footage': ['sum']})
df

Square Footage
                                                           sum
Agency Name                       Year Acquired               
Appellate Court / Fifth District  1857.0               15124.0
Appellate Court / Fourth District 1967.0               16400.0
Appellate Court / Second District 1967.0               43330.0
Appellate Court / Third District  1858.0               15000.0
                                  1860.0                3700.0
...                                                        ...
Western Illinois University       2007.0                9732.0
                                  2010.0                1200.0
                                  2012.0               60300.0
                                  2014.0              264870.0
                                  2015.0              106120.0

[1171 rows x 1 columns]

In [242]:
df = buildings.groupby(['Agency Name', 'Year Acquired']).agg({'Square Footage': ['sum']}).reset_index()
df.columns = ['Agency Name', 'Year Acquired', 'Square Footage'] #.loc('Western Illinois University') #['Agency Name'].unique()
df.groupby('Agency Name').get_group('Western Illinois University')[['Year Acquired', 'Square Footage']].head()
#df.groupby('Agency Name')[['Year Acquired', 'Square Footage']].head()

# what im doing here is finding a convoluted way to parse through data to get info for the agency name



,Year Acquired,Square Footage
1149,1956.0,198190.0
1150,1962.0,258408.0
1151,1963.0,121368.0
1152,1965.0,135200.0
1153,1966.0,270388.0


In [243]:
# checking to see if we can make a line plot using the data from one agency name (i.e., Department of Natural Resources)
# Agency: Department of Natural Resources (DNR)
DNR = buildings.groupby('Year Acquired').get_group('Department of Natural Resources')
DNR

KeyError: 'Department of Natural Resources'

I have been thinking about this kinda wrong. i've been focusing in linking the line and grid by using the agency name. For some reason i was focusing on it since it's the most finite or minute value. But only agencies only have one year acquired so only one data point which messes the line graph. 

but we can make a line graph using the congress district selected in the grid heat map. If we organize things by congress district we can get multiple values
let's try organizing by congress dist

In [244]:
# when i try to print both i and j j turns out not defined but i is fine??? why?
#print(i, j)

In [245]:
i

18

## Combining both figures into a dashboard

In [246]:
# 1. data --  buildings_heatmap
buildings_heatmap = pd.pivot_table(buildings, index=["Congress Dist"],values=["Square Footage"],
                                 columns=["Agency Name"],
                                  aggfunc=np.sum)
color_data = buildings_heatmap.values
congress_dist = buildings_heatmap.index # or buildings['Congress Dist'].unique()
agency_name = buildings_heatmap.columns.levels[1].to_list()  # or buildings['Agency Name'].unique()

# 2. scales
# 2.1 color scale
col_sc = bqplot.ColorScale(scheme='Blues')
# 2.2 x/y scales
x_sc = bqplot.OrdinalScale() # categorical
y_sc = bqplot.OrdinalScale()

# 3. Axis
# 3.1 color axis means colorbar
c_ax = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right')
# 3.2 x/y axis
x_ax = bqplot.Axis(scale=x_sc)
y_ax = bqplot.Axis(scale=y_sc, orientation='vertical')

# 4. Mark
heat_map = bqplot.GridHeatMap(color=np.log10(color_data), row=congress_dist, column=agency_name , scales={'color':col_sc, 'row':y_sc, 'column':x_sc},
                              interactions={'click':'select'}, # if i click on a bin it selects that bin
                              anchor_style={'fill':'red'}, # this highlights our selection to the color red
                              selected_style={'fill':'red'}, # changes our selected box a bit
                              unselected_style={'opacity':1.0} # a lil see through for the not selected ones
                              )


# 5. Interaction ... adding to the 'marks' w/ interactions
def on_selected(change):
    if len(change['owner'].selected) == 1:
        i, j = change['owner'].selected[0] # why is index 0 here? wouldnt this make i and j the same value as the row rather than the row and column index of the grid
        #i, j = heat_map.selected[0]
        # i represents that congress dist/ row and j is the agency index number
        v = buildings_heatmap.iloc[i,j] # value of the quare footage in that location
        mySelectedLabel.value = 'Square footage: ' + str(v) + '| Agency Name: ' + buildings_heatmap.columns[i][1] + '| Congress District: ' + str(i) 
        df = buildings.groupby(['Congress Dist']).get_group(i)[['Year Acquired', 'Square Footage']]
        lines.x = df.groupby('Year Acquired').sum().index
        lines.y = df.groupby('Year Acquired').sum().values
        line_fig.title = 'Total Square Footage Acquired in Congress District ' + str(i)
        
heat_map.observe(on_selected, 'selected') # linking selected trait of the heat_map with on_selected function

# 6. Figure
fig = bqplot.Figure(title='Congressional District and Agency Name Heat Map', axes= [c_ax, x_ax, y_ax], marks=[heat_map])


myDashboard = ipywidgets.VBox([mySelectedLabel, fig])
myDashboard

In [247]:
# lineplot
# 1. data

# total_square_footage = buildings.groupby('Year Acquired')['Square Footage'].sum()
df = buildings.groupby(['Congress Dist']).get_group(i)[['Year Acquired', 'Square Footage']]
# df is a sub sect dataframe of the selected congress district
total_square_footage = df.groupby('Year Acquired').sum()

# 2. Scales

x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

# 3. Axes

ax_x = bqplot.Axis(scale=x_sc, label='Year Acquired')
ax_y = bqplot.Axis(scale=y_sc, label='Total Square Footage', orientation='vertical')


# 4. Marks

lines = bqplot.Lines(x=total_square_footage.index, y=total_square_footage.values, scales={'x':x_sc, 'y':y_sc})

# 5. Interactivity


# 6. Figure
line_fig = bqplot.Figure(title='Total Square Footage Acquired in Congress District ' + str(i), marks=[lines], axes=[ax_x, ax_y])
line_fig



Figure(axes=[Axis(label='Year Acquired', scale=LinearScale()), Axis(label='Total Square Footage', orientation=…

In [248]:
figures = ipywidgets.HBox([fig, line_fig])
myDashboard = ipywidgets.VBox([mySelectedLabel, figures])

myDashboard